# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли цель получения кредита, уровень дохода, семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Исследовательский анализ данных ввключает в себя:

- обзор данных и поиск аномалий
- обработку пропусков
- обработку  дубликатов
- лемматизацию значений
- категоризацию данных
- поиск завизисимостей

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Загрузка необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

Загрузка данных и изучение общей информации

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06



Таблица состоит из 12 столбцов и включает в себя 21525 строк. Количество значений в столбцах различается, это говорит о том, что в столбцах `days_employed` и `total_income` по 2174 пропущенных значения.

По типу данных, столбцы можно разделить на три категории:
- int64: `children`, `dob_years`, `education_id`, `family_status_id`, `debt`
- float64: `days_employed`, `total_income`
- object: `education`, `family_status`, `gender`, `income_type`, `purpose`

Выведем 10 случайных строк таблицы

In [4]:
data.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8666,2,-1152.019631,42,среднее,1,женат / замужем,0,M,сотрудник,0,259048.825619,операции с недвижимостью
12020,0,-980.887522,56,высшее,0,женат / замужем,0,F,сотрудник,0,105294.302465,покупка недвижимости
14688,1,-148.516486,48,среднее,1,женат / замужем,0,F,сотрудник,0,50160.795209,получение высшего образования
19455,0,-2394.069195,45,среднее,1,женат / замужем,0,F,компаньон,0,491886.433062,на покупку автомобиля
17417,2,-2576.727516,29,Высшее,0,женат / замужем,0,F,сотрудник,0,213057.678703,дополнительное образование
18477,0,-1573.326646,25,неоконченное высшее,2,Не женат / не замужем,4,F,компаньон,0,134120.398957,покупка коммерческой недвижимости
20140,0,NaN,36,высшее,0,гражданский брак,1,F,компаньон,0,NaN,сыграть свадьбу
17864,0,371113.904423,57,среднее,1,вдовец / вдова,2,F,пенсионер,0,257369.340734,заняться образованием
3376,2,-3675.321201,32,среднее,1,Не женат / не замужем,4,F,сотрудник,0,97778.008403,операции с коммерческой недвижимостью
5645,0,362122.455370,65,среднее,1,женат / замужем,0,F,пенсионер,0,82137.734574,на покупку подержанного автомобиля



Разберём, какую информацию содержат столбцы:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита
 

**Вывод**

Каждая строка таблицы содержит информацию о заёмщике. Для дальнейшего анализа необходимо решить проблему пропусков в столбцах `days_employed` и `total_income`, проверить корректность формата данных и избавиться от дупликатов. Для оценки способности потенциального заёмщика вернуть кредит особенно ценны столбцы `debt`, `children`, `family_status`, `total_income`.

## Шаг 2. Предобработка данных

### Обработка пропусков

Перед удалением пропусков и дубликатов необходимо проверить, все ли данные предствленны в корректном виде.

Получим уникальные значения и их количество для столбца 'children':

In [5]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Число детей не может быть отрицательным, поэтому перезапишем эти значения равными нулю. Слишком большое для количество детей число 20, вероятно, явлеется ошибкой ввода и соответствует 2. Методом replace() изменим некорректные значения:

In [6]:
data['children'] = data['children'].replace(-1, 0)
data['children'] = data['children'].replace(20, 2)

data['children'].value_counts()

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

На первый взгляд в столбце `days_employed` положительные значения слишком большие, чтобы быть днями трудового стажа. А значения, которые могут соответствовать реальным значениям трудового стажа в днях - отрицательные. Проверим, вызвав минимальное положительное значение, и минимальное отрицательное значение(соответствующее максимальному общему трудовому стажу):

In [7]:
print('Минимальное положительное значение в столбце "days_employed": ', data.loc[(data['days_employed'] > 0) & (data['days_employed'].isnull() == False)]['days_employed'].min())
print('Минимальное отрицательное значение в столбце "days_employed": ', data.loc[(data['days_employed'] < 0) & (data['days_employed'].isnull() == False)]['days_employed'].min())

Минимальное положительное значение в столбце "days_employed":  328728.72060451825
Минимальное отрицательное значение в столбце "days_employed":  -18388.949900568383


Проверим, есть ли связь между форматом данных в столбце `days_employed` и типом занятости заёмщика

In [8]:
data[data['days_employed'] > 0]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [9]:
data[data['days_employed'] < 0]['income_type'].value_counts()

сотрудник          10014
компаньон           4577
госслужащий         1312
предприниматель        1
студент                1
в декрете              1
Name: income_type, dtype: int64

Наблюдается зависимость, между типом занятости заёмщика и форматом данных в столбце `days_employed`. Трудовой стаж пенсионеров и безработных выражен в часах и имеет положительное значение, для остальных категорий стаж выражен в днях и имеет отрицательное значение.

Приведём данные к единому формату:

In [10]:
def day_employed_to_correct(days):
    """
    Принимает величину трудового стажа, возвращает модуль числа дней трудового стажа.
    Если значение положительное, то оно выражено в часах и функция делит число на 24.
    """
    if days > 0:
        # 
        days /= 24
    return abs(days)

data['days_employed'] = abs(data['days_employed'].apply(day_employed_to_correct))


Проверим, существуют ли в таблице заёмщики, возраст которых меньше 18 лет.

In [11]:
data[data['dob_years'] < 18]['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

101 Заёмщик имеет в столбце возраст имеет значение 0. Заменим его медианным значением по столбцу `dob_years`

In [12]:
def to_median_value(column, value):
    """Принимает столбец и значение, возвращает ячейкам с совпадающим значением медиану по столбцу."""
    column_median = column.median()
    return column.replace(value, column_median)


data['dob_years'] = to_median_value(data['dob_years'], 0)

Проверим, существуют ли некорректные значения в столбце `gender`

In [13]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Изменим значение XNA на то, что встречается чаще.

In [14]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'

Проверим уникальные значения в столбце `debt`:

In [15]:
data['debt'].unique()

array([0, 1])

Количество пропусков в столбцах `days_employed` и `total_income` совпадает. Посчитаем количество строк, содержащих пропуски в этих столбцах.

In [16]:
data.loc[data['days_employed'].isnull() & data['total_income'].isnull()]['children'].count()

2174

Количество строк, содержащих пропуски в столбцах `days_employed` и `total_income` совпадает с числом пропусков, обнаруженным вызовом метода info(). Выведем 10 строк, содержащих пропуски, для  поиска закономерности.

In [17]:
data.loc[data['days_employed'].isnull() & data['total_income'].isnull()].sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
19470,0,NaN,48,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,жилье
16019,3,NaN,29,среднее,1,гражданский брак,1,M,сотрудник,0,NaN,покупка недвижимости
10413,1,NaN,51,среднее,1,женат / замужем,0,F,компаньон,0,NaN,покупка жилой недвижимости
1725,0,NaN,52,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции со своей недвижимостью
16410,0,NaN,58,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
5617,0,NaN,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,NaN,строительство собственной недвижимости
18715,0,NaN,41,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,покупка жилья для сдачи
14933,2,NaN,36,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,получение высшего образования
4182,1,NaN,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,NaN,свадьба
6541,0,NaN,29,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,операции с недвижимостью


Помимо пропусков в столбцах о доходе и трудовом стаже, заёмщики не имеют ничего общего - они разных возрастов, имеют разный тип занятости и уровень образования, пол, возраст, семейное положение и количество детей. Вероятно, пропуски возникли из-за ошибки заполнения данных о заемщиках.

Пропущенным значениям в столбцах `days_employed` и `total_income` присвоим медианное значение по столбцу:

In [18]:
def null_to_median_value(column):
    '''Принимает столбец, возвращает пропущенным значениям медиану по столбцу'''
    column_median = column.median()
    return column.fillna(column_median)

data['days_employed'] = null_to_median_value(data['days_employed'])
data['total_income'] = null_to_median_value(data['total_income'])

Получим общую информацию о таблице:

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Представление данных в столбце `days_employed` приведено к единому формату. Примечательно, что формат данных отличался для заёмщиков с типом занятости "пенсионер" и "безработный".  Пропуски в столбцах `days_employed`, `total_income` и `dob_years` заполнены медианными значениями. Пропуск в столбце `gender` заполнен наиболее часто встречаемым значением. Исправлены значения в столбцах `children` и `family_status`.

### Замена типа данных

Определим, сколько памяти занимает датасет и максимальные значения по столбцам, чтобы определить, как изменить тип данных:

In [20]:
memory_usage = data.memory_usage().sum()
print(memory_usage)
data.max()

2066528


children                          5
days_employed               18388.9
dob_years                        75
education            ученая степень
education_id                      4
family_status       женат / замужем
family_status_id                  4
gender                            M
income_type                 студент
debt                              1
total_income             2.2656e+06
purpose             сыграть свадьбу
dtype: object

Количество детей, возрат заёмщиков, индексы уровня образования и семейного положений не имеют отрицательных значений и не превышают 255, поэтому целесообразно присвоить этим значениям тип `uint8`. Количество дней трудового стажа не превышает 65535 и должно быть представлено натуральным числом, следует заменить его тип на `uint16`.
  
Ежемесячный доход, судя по максимальному значению, можно представить типом `float32`.
  
Значения в столбце `debt` принимают значения 1 или 0, а следовательно их можно представить типом `bool`.

Изменим тип данных для обозначенных столбцов и определим, какой объем памятти используют данные:

In [21]:
data[['children', 'dob_years', 'education_id', 'family_status_id']] = data[['children', 'dob_years', 'education_id', 'family_status_id']].astype(np.uint8)
data['days_employed'] = data['days_employed'].astype(np.uint16)
data['total_income'] = data['total_income'].astype(np.float32)
data['debt'] = data['debt'].astype(np.bool)

Узнаем, удалось ли оптимизировать хранение данных и сколько памяти удалось сэкономить:

In [22]:
memory_usage = 1 - data.memory_usage().sum() / memory_usage
print('{:.1%}'.format(memory_usage))

46.9%


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null uint8
days_employed       21525 non-null uint16
dob_years           21525 non-null uint8
education           21525 non-null object
education_id        21525 non-null uint8
family_status       21525 non-null object
family_status_id    21525 non-null uint8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null bool
total_income        21525 non-null float32
purpose             21525 non-null object
dtypes: bool(1), float32(1), object(5), uint16(1), uint8(4)
memory usage: 1.0+ MB


**Вывод**

При замене типов данных следует обращать внимание на то, какие значения эти данные принимают и какой смысл несут. Замена первоначальных типов данных происходила в зависимости от их характера позволила сэкономить почти 47% от первоначального объёма памяти. 

### Обработка дубликатов

Дубликаты в таблице могут выглядеть как строки с идентичной информацией, такая проблема возникает при ошибочном повторном копировании данных. Ещё дубликаты могут быть неявными, из-за одинаковых по смыслу, но разных по названию категорий. Такая проблема возникает при работе нескольких пользователей с одной таблицей и отсутствия ограничений для значений категорий.

Поиск количества дублирующихся строк в таблице:

In [24]:
print('Дубликатов в таблице: ', data.duplicated().sum())

Дубликатов в таблице:  55


Для пар столбцов со строковыми значениями и индексами (`education` и `education_id`, `family_status` и `family_status_id`) проверим сооответствие числа уникальных значений.

In [25]:
def unique_values_unique_id(series_values, series_id):
    """
    Принимает два столбца и сравненивает количество уникальных значений в столбцах.
    Возвраащает сообщение о соответствии/несоответствии и списо значений столбца series_values.
    """   
    if len(series_values.unique()) == len(series_id.unique()):
        return print('количество индексов соответствует количеству значений', series_values.unique(), sep='\n')
    else:
        return print('количество индексов не соответствует количеству значений', series_values.unique(), sep='\n')   


Проверка сответствия числа уникальных значений в столбце 'education' числу значений в столбце 'education_id'

In [26]:
unique_values_unique_id(data['education'], data['education_id'])

количество индексов не соответствует количеству значений
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


Из-за написания данных об уровне образования в разном регистре могут возникнуть замаскированные повторы. Необходимо привести данные к единому формату - в нижнем регистре.

In [27]:
data['education'] = data['education'].str.lower()

Проверка сответствия числа уникальных значений в столбце 'education' числу значений в столбце 'education_id'

In [28]:
unique_values_unique_id(data['education'], data['education_id'])

количество индексов соответствует количеству значений
['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Проверка сответствия числа уникальных значений в столбце 'family_status' числу значений в столбце 'family_status_id'

In [29]:
unique_values_unique_id(data['family_status'], data['family_status'])

количество индексов соответствует количеству значений
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


В столбце о семейном положении нет дублирующихся категорий, но следует привести данные к единому формату - в нижнем регистре

In [30]:
data['family_status'] = data['family_status'].str.lower()

Проверим количество дубликатов в таблице

In [31]:
print('Дубликатов в таблице: ', data.duplicated().sum())

Дубликатов в таблице:  72


Количество дубликатов, найденных связкой методов duplicated().sum() увеличилось, так как в столбце `education` значения были записаны в разном регистре. Удалим дубликаты с методом reset_index():

In [32]:
data = data.drop_duplicates().reset_index(drop = True)

**Вывод**

Дубликаты могут возникать из-за работы нескольких пользователей с одной таблицей, отсутствия ограничений для значений категорий, создания составной таблицы из других таблиц. В таблице с информацией о заёмщиках удалено 72 дубликата. Из-за написания уровня образования в разном регистре, первоначально методами duplicated().sum() удалось обнаружить только 55 повторяющихся строк. Данные в столбцах `education` и `family_status` приведены к нижнему регистру.

### Лемматизация

Для того, чтобы определить категории для столбца `purpose`, необходимо произвести лемматизацию данных.

In [33]:
def lemma(text):
    """Принимает в качестве параметра текст и возвращает список лемм."""
    lemma = m.lemmatize(text)
    return lemma

Создание столбца, содержащего список лемм для данных из столбца о целях получения кредита

In [34]:
data['purpose_lemma'] = data['purpose'].apply(lemma)

Вызов метода Counter для подсчета частоты значений

In [35]:
Counter(data['purpose_lemma'].sum()).most_common()

[(' ', 33568),
 ('\n', 21453),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2323),
 ('свой', 2230),
 ('на', 2221),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 767),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

**Вывод**

Чтобы категоризировать столбец `purpose`, необходимо выделить часто встречающиеся цели получения кредита. Решить эту задачу может метод лемматизации и последующий подсчет частоты использования лемм. Полученные леммы позволяют выделить следующие категории: недвижимость(недвижимость, жильё), автомобиль, образование, свадьба. Цели, которые не попадают в обозначенные категории отнесём в общую категорию "другое".

### Категоризация данных

Сформируем категории для количества детей, месячного дохода и цели полувчения кредита:

In [36]:
def children_category(children):
    """Возвращает категорию, в зависимости от количества детей."""
    if children < 3:
        return str(children)
    return 'многодетный'

Формирование столбца с категориями количества детей:

In [37]:
data['children_category'] = data['children'].apply(children_category)

In [38]:
def income_category(total_income):
    """Возвращает категорию, в зависимости от месячного дохода"""
    if total_income < 100000:
        return 'до 100000'
    elif total_income < 200000:
        return 'от 100000 до 200000'
    elif total_income < 300000:
        return 'от 200000 до 300000'
    elif total_income < 400000:
        return 'от 300000 до 400000'
    return 'более 400000'

Формирование столбца с категориями месячного дохода:

In [39]:
data['income_category'] = data['total_income'].apply(income_category)

In [40]:
def purpose_category(purpose_lemma):
    """Принимает спиок лемм, возвращает категорию в зависимости от соответствия условиям."""
    if 'жилье' in purpose_lemma or 'недвижимость' in purpose_lemma:
        return 'недвижимость'
    elif 'автомобиль' in purpose_lemma:
        return 'автомобиль'
    elif 'образование' in purpose_lemma:
        return 'образование'
    elif 'свадьба' in purpose_lemma:
        return 'свадьба'
    else:
        return 'другое'

Формирование столбца с категориями целей получения кредита:

In [41]:
data['purpose_category'] = data['purpose_lemma'].apply(purpose_category)

Создадим справочники для категориальных переменных:

In [42]:
children_dict = data[['children_category']].drop_duplicates().reset_index(drop = True)   
children_dict

,children_category
0,1
1,0
2,многодетный
3,2


Создадим справочник для категорий уровня образования:

In [43]:
education_dict = data[['education', 'education_id']].drop_duplicates().reset_index(drop = True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Создадим справочник для категорий семейного статуса:

In [44]:
family_status_dict = data[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop = True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Создадим справочник справочник для категорий пола:

In [45]:
gender_dict = data[['gender']].drop_duplicates().reset_index(drop = True)
gender_dict

,gender
0,F
1,M


Создадим справочник для категорий типа занятости:

In [46]:
income_type_dict = data[['income_type']].drop_duplicates().reset_index(drop = True)
income_type_dict

,income_type
0,сотрудник
1,пенсионер
2,компаньон
3,госслужащий
4,безработный
5,предприниматель
6,студент
7,в декрете


Создадим справочник для категорий целей получения кредита:

In [47]:
purpose_dict = data[['purpose_category']].drop_duplicates().reset_index(drop = True)
purpose_dict

,purpose_category
0,недвижимость
1,автомобиль
2,образование
3,свадьба


**Вывод**

Для анализа зависимости задолженности по кредиту, для данных о количестве детей, ежемесячном доходе и цели получения кредита сформированы категории. Для всех столбцов с категориальными значениями созданы таблицы-справочники.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Формирование сводной таблицы, с группировкой по столбцу `children_category`:

In [48]:
children = data.pivot_table(index = 'children_category', values = 'debt', aggfunc = ['count', 'mean'])
children.columns = ['count', '%']
children = children.style.format({'%': '{:.2%}'})
children

,count,%
children_category,,
0,14137,7.53%
1,4808,9.23%
2,2128,9.49%
многодетный,380,8.16%


**Вывод**

Заёмщики с детьми чаще имеют задолженность по кредиту, по сравнению с бездетными. Многодетные заёмщики имеют меньший процент задолженности по сравнению с теми, у кого 1 или 2 ребенка, что можно объяснить дополнительными льготами для семей с тремя и более детей. Чаще всего просрочивали выплату по кредиту заёмщики с двумя детьми, так как они имеют меньше льгот по сравнению с категорией "многодетные", но больше непрогнозируемых трат, чем заёмщики с одним ребёнком. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Формирование сводной таблицы, с группировкой по столбцу `family_status`:

In [49]:
family_status = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count', 'mean'])
family_status.columns = ['count', '%']
famyly = family_status.style.format({'%': '{:.2%}'})
famyly

,count,%
family_status,,
в разводе,1195,7.11%
вдовец / вдова,959,6.57%
гражданский брак,4150,9.35%
женат / замужем,12339,7.55%
не женат / не замужем,2810,9.75%


**Вывод**

Чаще остальных имеют задолженность по кредиту заёмщики, не состоящие сейчас и ранее в зарегистрированных отношениях. Заёмщики, состоящие в браке, чаще ведут общий бюджет и благодаря этому реже имеют задолженность в случае непредвиденных трат.     

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Формирование сводной таблицы, с группировкой по столбцу `income_category`:

In [50]:
income = data.pivot_table(index = 'income_category', values = 'debt', aggfunc = ['count', 'mean'])
income.columns = ['count', '%']
income = income.sort_values(by = '%').style.format({'%': '{:.2%}'})
income

,count,%
income_category,,
более 400000,528,5.87%
от 200000 до 300000,3584,7.03%
от 300000 до 400000,954,7.86%
до 100000,4463,7.93%
от 100000 до 200000,11924,8.63%


**Вывод**

Из полученных от банка данных не ясно, как соотносится размер кредита, ежемесячный платеж и ежемесячный доход. Заёмщики с высоким уровнем дохода чаще остальных выплачивали кредит в срок, а большинство должников имеют месячный доход от 100000 до 200000. Для остальных категорий процент задолженностей не сильно отличается.

- Как разные цели кредита влияют на его возврат в срок?

Формирование сводной таблицы, с группировкой по столбцу `purpose_category`:

In [51]:
purpose = data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['count', 'mean'])
purpose.columns = ['count', '%']
purpose = purpose.style.format({'%': '{:.2%}'})
purpose

,count,%
purpose_category,,
автомобиль,4306,9.36%
недвижимость,10811,7.23%
образование,4013,9.22%
свадьба,2323,8.01%


**Вывод**

Самой частой целью получения кредита являются операции с недвижемостью, заемщики из этой категории чаще возвращают кредит в срок. Заёмщики, чьей целью получения кредита были операции с автомобилем и получение образования, чаще остальных имели задолженность по кредиту. Если автомобиль приобретался как средство заработка, то в случае поломки заёмщик несёт не только затраты на ремонт, но и не имеет средств к выплате кредита. Получение образования не гарантирует трудоустройсво, а значит заемщик может не иметь средств выплачивать кредит в срок. Заемщики, берущие кредит на проведение свадьбы, обычно планирует выплачивать кредит из подаренных средств. Вероятно, что эта стратегия работает в ~92% случаев.

## Шаг 4. Общий вывод

***Рабочая гипотеза:*** На возврат кредитав в срок влияют количество детей заёмщика, семейное положение, ежемесячный доход, цель получения кредита.

Был проведен исследовательский анализ данных о платежеспособности заёмщиков, полученных от банка. 
В предоставленных данных обнаружены и обработаны пропуски. Некорректные значения исправлены, выявлены и удалены дубликаты. Для поиска зависимости того, как на возврат кредита без задолженности влияют количество детей,  семейное положение, доход и цель получения была проведена категоризация данных. Так как цели получения кредита имеют широкий спектр значений, для категоризации этого фактора была проведена лемматиация, что позволило выделить следующие категории: недвижимость, автомобиль, образование, свадьба.

Влияние количества детей на возврат кредита в срок:

- реже остальных задерживали выплату заёмщики без детей - в 7.53% случаев
- заёмщики с одним ребенком оказывались в должниках в 9.23% случаев, с двумя детьми - в 9.49%
- многодетные заёмщики просрочивали выплаты в 8.16% случаев

Влияние семейного положения:

- не состоящие в браке заёмщики опыздывали с платежом в 9.75% случаев
- состоящие в гражданском браке - в 9.35 % случаев
- состоящие в зарегистрированном браке становились должниками в 7.55% случаев
- среди заёмщиков в разводе должников 7.11%
- овдовевшие заёмщики не возвращили кредит в рок лишь в 6.75% случаев

Влияние уровня дохода:

- чаще остальных оказывались в должниках заёмщики с ежемесячным доходом от 100000 до 200000 - 8.63%
- меньше всего должников среди заёмщиков с доходом выше 400000 - 5.87%
- должники среди заёмщиков с доходом до 100000 - 7.93%
- должники среди заёмщиков с доходом от 200000 до 300000 - 7.03%
- должники среди заёмщиков с доходом от 300000 до 400000 - 7.86%

Влияние цели получения кредита на задолженность:

- чаще остальных задерживали выплату по кредиту те заёмщики, чьей целью были операции с атовомобилем - 9.36% и образование - 9.22%
- заёмщики, оформляющие получение кредита на проведение свадьбы, оказывались в должникав в 8.01% случаев
- заёмщики, оформляющие кредит на проведение операций с недвижимостью, задерживали выплату по кредиту лишь в 7.23% случаев.

Наибольшую разницу между максимальным и минимальным числом должников по категориям имеет фактор семейного положения - 3%. В этом факторе выделена категорияс наибольшим числом должников:  "не женат / не замужем" - 9.75%.